In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from functools import wraps
from inspect import signature
import inspect

In [37]:
def scaling(df, how:list = ["minmax", "standard"]):
    if how == "minmax":
        scaler = MinMaxScaler
        return scaler.fit_transform(df)
    elif how == "standard":
        scaler = StandardScaler
        return scaler.fit_transform(df)

In [38]:
from functools import wraps
from inspect import signature

    
def param_info(func):
    sig = signature(func)
    for param in sig.parameters.values():
        print(param.name)
        print(' -', param.default)
        print(' -', param.kind)


def safe_param(func):
    ok_args = False
    ok_kwargs = False
    
    list_params = []
    keyword_params = set()
    
    sig = signature(func)
    for param in sig.parameters.values():
        if param.kind == param.VAR_POSITIONAL:
            ok_args = True
        if param.kind == param.VAR_KEYWORD:
            ok_kwargs = True
            
        if param.kind in [param.POSITIONAL_OR_KEYWORD]:
            list_params.append(param.name)
        if param.kind in [param.POSITIONAL_OR_KEYWORD, param.KEYWORD_ONLY]:
            keyword_params.add(param.name)
            
    def get_default_value(param_name):
        original = sig.parameters[param_name]
        no_default = original.default is original.empty
        return None if original.default is original.empty else original.default
            
    @wraps(func)
    def wrap(*args, **kwargs):
        if not ok_args:
            args = args[:len(list_params)]
        
        if not ok_kwargs:
            temp = {k: v for k, v in kwargs.items() if k in keyword_params}
            kwargs = temp
        
        if len(args) < len(list_params):
            not_set_list_params = list_params[len(args):]
            for param in not_set_list_params:
                if param in kwargs:
                    continue
                    
                kwargs[param] = get_default_value(param)
        
        not_set_keyword_params = keyword_params - set(list_params) - set(kwargs.keys())
        for param in not_set_keyword_params:
            kwargs[param] = get_default_value(param)
        
        return func(*args, **kwargs)
    return wrap

In [11]:
param_info(scaling)

df
 - <class 'inspect._empty'>
 - POSITIONAL_OR_KEYWORD
how
 - ['minmax', 'standard']
 - POSITIONAL_OR_KEYWORD


In [12]:
safe_param(scaling)

<function __main__.scaling(df, how=['minmax', 'standard'])>

In [20]:
inspect.Parameter(scaling, kind=POSITIONAL_ONLY)

NameError: name 'POSITIONAL_ONLY' is not defined

In [43]:
def param_info(func):
    sig = signature(func)
    for param in sig.parameters.values():
        print(param.name)
        print(' -', param.default)
        print(' -', param.kind)
        print(' -', param.annotation)



In [44]:
param_info(scaling)

df
 - <class 'inspect._empty'>
 - POSITIONAL_OR_KEYWORD
 - <class 'inspect._empty'>
how
 - ['minmax', 'standard']
 - POSITIONAL_OR_KEYWORD
 - <class 'list'>
